In [94]:
f = open("example.txt", "r")
lines = [line.rstrip() for line in f]

In [95]:
class Directory:
    def __init__(self, name, parent):
        self.name = name
        self.contents = [] # contains a mix of Files and other Directories
        self.parent = parent
    def __repr__(self):
        if self.parent != None: # if we are not the root directory
            return repr(self.parent) + "|" + self.name
        else: return self.name

class File:
    def __init__(self, name, size, parent: Directory):
        self.name = name
        self.size = size
        self.parent = parent
    def __repr__(self):
        return self.parent + "|" + self.name

In [96]:
def findDir(name, parent: Directory): # returns a Directory with given name in parent Directory.  Returns None if none found
    for item in parent.contents:
        if isinstance(item, Directory):
            if item.name == name:
                return item
    return None

In [97]:
currentDir = Directory("/", None)
path = [] # a Stack of Directories.  does not include current Directory

i = 0
while i < len(lines):
    print(currentDir, end=" ")
    if currentDir.parent != None:
        print(currentDir.parent.name)
    else: print()

    line = lines[i]
    if line[0] == "$":
        command = line.split()[1]

        if command == "cd":
            dirName = line.split()[2]

            if dirName != "..":
                path.append(currentDir)

                # Check if the Directory we are cd-ing into already exists
                if findDir(dirName, path[-1]) == None: # if not, create it
                    currentDir = Directory(dirName, path[-1])
                else: # otherwise, set that pre-existing Directory as our current
                    currentDir = findDir(dirName, path[-1]) 

            else: # we are cd-ing upward
                currentDir = path.pop()
                i += 1
                continue

        if command == "ls":
            i += 1
            continue

    if line[0] != "$":
        if line.split()[0] != "dir": # add the file size and name to our list of current Directory's contents
            currentDir.contents.append(File(line.split()[1], int(line.split()[0]), currentDir))
        else:
            currentDir.contents.append(Directory(line.split()[0], currentDir))
    
    i += 1


/ 
/|/ /
/|/ /
/|/ /
/|/ /
/|/ /
/|/ /
/|/|a /
/|/|a /
/|/|a /
/|/|a /
/|/|a /
/|/|a /
/|/|a|e a
/|/|a|e a
/|/|a|e a
/|/|a /
/|/ /
/|/|d /
/|/|d /
/|/|d /
/|/|d /
/|/|d /


In [98]:
print(currentDir)

/|/|d
